In [2]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW
from utils import tokenize_function, TextDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

## Create Data Set

In [2]:
'''
Format of train/val should be:
Where each dictionary is a data input, with input and output
data = [
    {"input": "FOOD AWADAWDWA DOG 1FD@#$ RUTROW DEIXIOUS", "output": "DELICIOUS DOG FOOD"},
    {"input": "CHCOLATE BAR CACAO 70%", "output": "CHOCOLATE BAR CACAO 70%"},
    # Add more examples...
]
'''
# FOR GARY
# TODO
# Take public data set and convert to list of dictionaries (see below for data examples)
# data = 

'''
SUGGESTIONS:

Get a data set of product names. 
For example: https://www.kaggle.com/datasets/promptcloud/walmart-product-details-2020/data

1. Create a name scrambler function.
This function takes each word in the product name and scrambles both the position in the name and applies a MINOR scramble to the letters

2. Loop through product data sets, For each product name
- Create multiple data inputs
    - product name is output
    - scramble the product name multiple times to create multiple inputs

i.e for product name: "Whiskers Cat Food", you could produce 3 data inputs:
{"input": "CST FOOL WHISKERS", "output": "Whiskers Cat Food"},
{"input": "TAC WISKERS FEEDS", "output": "Whiskers Cat Food"},
{"input": "WFHSIEKRS CAT FOPD", "output": "Whiskers Cat Food"},

Create more than 3 outputs though
'''


'\nSUGGESTIONS:\n\nGet a data set of product names. \nFor example: https://www.kaggle.com/datasets/promptcloud/walmart-product-details-2020/data\n\n1. Create a name scrambler function.\nThis function takes each word in the product name and scrambles both the position in the name and applies a MINOR scramble to the letters\n\n2. Loop through product data sets, For each product name\n- Create multiple data inputs\n    - product name is output\n    - scramble the product name multiple times to create multiple inputs\n\ni.e for product name: "Whiskers Cat Food", you could produce 3 data inputs:\n{"input": "CST FOOL WHISKERS", "output": "Whiskers Cat Food"},\n{"input": "TAC WISKERS FEEDS", "output": "Whiskers Cat Food"},\n{"input": "WFHSIEKRS CAT FOPD", "output": "Whiskers Cat Food"},\n\nCreate more than 3 outputs though\n'

In [3]:
# example list
data = [
    {"input": "FOOD AWADAWDWA DOG 1FD@#$ RUTROW DEIXIOUS", "output": "DELICIOUS DOG FOOD"},
    {"input": "CHCOLATE BAR CACAO 70%", "output": "CHOCOLATE BAR CACAO 70%"},
    {"input": "TOMTO SCE KETHCUP", "output": "TOMATO SAUCE KETCHUP"},
    {"input": "PNAPL JLCE NATURL", "output": "PINEAPPLE JUICE NATURAL"},
    {"input": "ALMOND MLIK UNSWEETND", "output": "ALMOND MILK UNSWEETENED"},
    {"input": "PEANUT BTER CRMCHY", "output": "PEANUT BUTTER CRUNCHY"},
    {"input": "IC CREAM VANILA FLAVOR", "output": "ICE CREAM VANILLA FLAVOR"},
    {"input": "APPLE SIDER VINIGR", "output": "APPLE CIDER VINEGAR"},
    {"input": "WHOL WHT BREAD SLICD", "output": "WHOLE WHEAT BREAD SLICED"},
    {"input": "ORANGE JLCE FRESH SQZD", "output": "ORANGE JUICE FRESH SQUEEZED"},
    {"input": "GRN TEA OGRNIC", "output": "GREEN TEA ORGANIC"},
    {"input": "SPGHTTI PASTA WHEAT", "output": "SPAGHETTI PASTA WHEAT"},
    {"input": "PRCHUTO SLCD", "output": "PROSCIUTTO SLICED"},
    {"input": "BRCOLI FRSH ORGANIC", "output": "BROCCOLI FRESH ORGANIC"},
    {"input": "CAROT JCE RAW", "output": "CARROT JUICE RAW"},
    {"input": "MZN HNY PRSSED", "output": "MANUKA HONEY PRESSED"},
    {"input": "STRAWBRY YOGRT LITE", "output": "STRAWBERRY YOGURT LIGHT"},
    {"input": "BLUERRYS FROZN", "output": "BLUEBERRIES FROZEN"},
    {"input": "SRIRCHA HOT SCE", "output": "SRIRACHA HOT SAUCE"},
    {"input": "ZUCCHN ORGNIC", "output": "ZUCCHINI ORGANIC"},
    {"input": "CHCKN BRST BNELESS", "output": "CHICKEN BREAST BONELESS"},
    {"input": "BTRSCOTCH CHIPS", "output": "BUTTERSCOTCH CHIPS"},
    {"input": "CRML SCE DIPPING", "output": "CARAMEL SAUCE DIPPING"},
    {"input": "CCHNUT WTR PURE", "output": "COCONUT WATER PURE"},
    {"input": "GREEK YOUGRT PLAIN", "output": "GREEK YOGURT PLAIN"},
    {"input": "WHT CHOCOLATE COOKS", "output": "WHITE CHOCOLATE COOKIES"},
    {"input": "SHRIMP CRCKERS SPICY", "output": "SHRIMP CRACKERS SPICY"},
    {"input": "GRN APPLES ORGNC", "output": "GREEN APPLES ORGANIC"},
    {"input": "RSPBRY JAM HOMEMADE", "output": "RASPBERRY JAM HOMEMADE"},
    {"input": "PEACH NECTAR CONCENTRTE", "output": "PEACH NECTAR CONCENTRATE"},
    {"input": "BBY SPINACH FRESH", "output": "BABY SPINACH FRESH"},
    {"input": "TMATO PASTE ORGNIC", "output": "TOMATO PASTE ORGANIC"},
    {"input": "GRLIC CLVES CRUSHED", "output": "GARLIC CLOVES CRUSHED"},
    {"input": "MANGO CHUNKS FROZN", "output": "MANGO CHUNKS FROZEN"},
    {"input": "HNY DRZZLE ORGANIC", "output": "HONEY DRIZZLE ORGANIC"},
    {"input": "LITE SPRAY OLIVE OIL", "output": "LIGHT SPRAY OLIVE OIL"},
    {"input": "CNMON POWDER PURE", "output": "CINNAMON POWDER PURE"},
    {"input": "BLAC PEPPER WHOLE", "output": "BLACK PEPPER WHOLE"},
    {"input": "SRIRCHA POWDER SPICY", "output": "SRIRACHA POWDER SPICY"},
    {"input": "POTATO CHPS CRISPY", "output": "POTATO CHIPS CRISPY"},
    {"input": "CRN CHPS CHEESY", "output": "CORN CHIPS CHEESY"},
]


In [4]:
train, val = train_test_split(data, test_size=0.1)

## Prep Training

In [5]:
# load model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [6]:
# create tokens for each data input
# tokenize_function input is (data set input, model tokenizer, and max length of string)
tokenized_train = [tokenize_function(i, tokenizer, 32) for i in train]
tokenized_val = [tokenize_function(i, tokenizer, 32) for i in val]

In [7]:
# create training set
train_dataset = TextDataset(tokenized_train)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [8]:
#create val set
val_dataset = TextDataset(tokenized_val)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)

## Run Training 

In [9]:
# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 10

#set model to training
model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # Training phase
    total_train_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        
        # Ensure the correct batch structure
        print(batch)
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        
        # Pass the inputs to the model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    
    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Training Loss: {avg_train_loss}")
    
    # Validation phase
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            labels = batch["labels"]
            
            # Pass the inputs to the model
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()
    
    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Validation Loss: {avg_val_loss}")
    
    # Set the model back to training mode for the next epoch
    model.train()


C:\Users\MSI\.conda\envs\reciept-scanner\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
{'input_ids': tensor([[    0, 14386,  2571,   673,   221,  2336,  6433,  6532, 17567,  2371,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   448,  1301,   487,   289,  9873,  4729,  8108,  1691,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   534, 30009,   530, 10540, 11621,   565, 12901, 33178,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   387,  6997,  3632,  3293,  3764,  3858, 33792,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     

In [10]:
print("FINAL RESULTS VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV")
model.eval()  # Set the model to evaluation mode
final_val_loss = 0
correct_predictions = 0
total_predictions = 0

correct_sentences = 0
total_sentences = 0

with torch.no_grad():
    for batch in val_dataloader:
        # Forward pass
        outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["labels"])
        loss = outputs.loss
        final_val_loss += loss.item()

        # Get model predictions (greedy decoding)
        predictions = outputs.logits.argmax(dim=-1)
        
        # Flatten the tensors for token-level comparison
        predictions_flat = predictions.view(-1)
        labels_flat = batch["labels"].view(-1)
        
        # Only consider non-padding tokens (ignore index 0 or whatever your pad token is)
        non_padding_mask = labels_flat != tokenizer.pad_token_id
        num_correct = (predictions_flat == labels_flat) & non_padding_mask
        
        correct_predictions += num_correct.sum().item()
        total_predictions += non_padding_mask.sum().item()

        # Calculate sentence-level accuracy
        for i in range(batch["input_ids"].size(0)):
            label_sentence = batch["labels"][i][non_padding_mask[i]].tolist()  # Get the label sentence excluding padding
            predicted_sentence = predictions[i][non_padding_mask[i]].tolist()  # Get the predicted sentence excluding padding
            
            if label_sentence == predicted_sentence:
                correct_sentences += 1
            total_sentences += 1

# Average Loss
avg_final_val_loss = final_val_loss / len(val_dataloader)
print(f"Final Validation Loss: {avg_final_val_loss}")

# token level accuracy
token_level_accuracy = correct_predictions / total_predictions
print(f"Final Token-Level Validation Accuracy: {token_level_accuracy:.4f}")

# sentence level accuracy
sentence_level_accuracy = correct_sentences / total_sentences
print(f"Final Sentence-Level Validation Accuracy: {sentence_level_accuracy:.4f}")

FINAL RESULTS VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Final Validation Loss: 1.3210673332214355
Final Token-Level Validation Accuracy: 0.7069
Final Sentence-Level Validation Accuracy: 0.0000


In [11]:
# save as pt 
torch.save(model.state_dict(), "bart.pt")

## Prediction example usage

In [12]:
# Load the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
# load weights
model.load_state_dict(torch.load("bart.pt"))
# Example input text for prediction
input_text = "FOOD AWADAWDWA DOG 1FD@#$ RUTROW DEIXIOUS"

# Tokenize the input
tokenized_input = tokenizer(input_text, return_tensors="pt", padding="max_length", max_length=32, truncation=True)

# Generate prediction
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    predicted_ids = model.generate(input_ids=tokenized_input["input_ids"], attention_mask=tokenized_input["attention_mask"], max_length=32)

# Decode the prediction
predicted_text = tokenizer.decode(predicted_ids[0], skip_special_tokens=True)
print(f"prediction: {predicted_text}")


prediction: FOOD AWADAWDWA DOG FOOD
